In [116]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import scanpy as sc
#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

In [66]:
import requests
from bs4 import BeautifulSoup

In [68]:
url = "https://data.nemoarchive.org/biccn/lab/zeng/transcriptome/scell/SSv4/mouse/raw/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

fastq_links = []  # Initialize an empty list to store the links

for link in soup.find_all('a'):
    href = link.get('href')
    if 'fastq' in href:
        print(href)
        fastq_links.append(href)  # Add the link to the list

# Optionally, print the entire list or do further processing
#print(fastq_links)

ProxyError: HTTPSConnectionPool(host='data.nemoarchive.org', port=443): Max retries exceeded with url: /biccn/lab/zeng/transcriptome/scell/SSv4/mouse/raw/ (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

In [50]:
os.chdir('/cndd/Public_Datasets/BICCN/BICCN_minibrain_data/data_freeze/smarter_cells/')
ko = pd.read_csv('sample_metadata.csv.gz')

In [51]:
ko

,Unnamed: 0,ar_id,exp_component_vendor_name,batch,batch_vendor_name,tube,tube_internal_name,tube_contents_nm,tube_contents_nm_from_vendor,tube_avg_size_bp,...,Injection_type,NewBatch,Vendor,MultiPlex,percent_reads_aligned_to_rrna_rmsk,percent_reads_aligned_to_trna_rmsk,percent_reads_aligned_to_ncrna,percent_reads_aligned_to_introns,percent_reads_aligned_to_intergenic,percent_reads_aligned_unique
0,LS-15395_S41_E1-50,543183476,LS-15395-41,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.162980,9.855384,3.264771,84.993663
1,LS-15395_S42_E1-50,543183474,LS-15395-42,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.281620,6.763949,2.856108,87.252036
2,LS-15395_S43_E1-50,543183472,LS-15395-43,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,14.448787,13.283785,4.473082,84.439773
3,LS-15395_S44_E1-50,543183470,LS-15395-44,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.687114,7.724415,3.083142,87.400895
4,LS-15395_S46_E1-50,543183466,LS-15395-46,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.844740,8.100115,3.478772,85.846808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,SM-GE92M_S092_E1-50,708586022,SM-GE92M-S092,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.692675,0.054178,4.007977,12.591947,27.800393,NaN
6296,SM-GE92M_S093_E1-50,708586372,SM-GE92M-S093,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.409496,0.051422,3.830221,9.094367,29.950267,NaN
6297,SM-GE92M_S094_E1-50,708586078,SM-GE92M-S094,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.610336,0.086938,3.419039,9.457431,29.709904,NaN
6298,SM-GE92M_S095_E1-50,708586242,SM-GE92M-S095,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.403770,0.040953,2.808983,6.679797,24.221576,NaN


In [60]:
os.chdir('/cndd/Public_Datasets/BICCN/BICCN_minibrain_data/data_freeze/snmc_and_snatac/metadata/')
ko = pd.read_csv('MOp_Metadata.tsv.gz', sep='\t')
ko

,cell,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,...,uid,BamFilteringRate,MappingRate,Pos96,Plate,Col96,Row96,Col384,Row384,Batch
0,2C_M_0,/gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,...,2C-CEMBA180409-2C-1-CEMBA180409-2C-2-A10,0.709307,0.655477,A10,CEMBA180409_2C_1,9,0,18,0,CEMBA180409_2C
1,2C_M_1,/gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,...,2C-CEMBA180409-2C-1-CEMBA180409-2C-2-A10,0.700327,0.659815,A10,CEMBA180409_2C_1,9,0,18,1,CEMBA180409_2C
2,2C_M_10,/gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,...,2C-CEMBA180409-2C-1-CEMBA180409-2C-2-A11,0.626633,0.684020,A11,CEMBA180409_2C_2,10,0,20,0,CEMBA180409_2C
3,2C_M_100,/gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,...,2C-CEMBA180409-2C-1-CEMBA180409-2C-2-B1,0.642258,0.685798,B1,CEMBA180409_2C_2,0,1,1,2,CEMBA180409_2C
4,2C_M_1000,/gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,...,2C-CEMBA180409-2C-3-CEMBA180409-2C-4-F8,0.649110,0.685797,F8,CEMBA180409_2C_3,7,5,15,11,CEMBA180409_2C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936,5D_M_995,/gale/raidix/rdx-4/CEMBA_RS1/5D/CEMBA180605_5D...,0.00605,0.80138,0.80017,0.03183,0.02594,800968.0,1767844,1190789.0,...,5D-CEMBA180605-5D-3-CEMBA180605-5D-4-G10,0.672636,0.673583,G10,CEMBA180605_5D_4,9,6,18,12,CEMBA180605_5D
9937,5D_M_996,/gale/raidix/rdx-4/CEMBA_RS1/5D/CEMBA180605_5D...,0.00476,0.80081,0.79986,0.02246,0.01778,718290.0,1594474,1064644.0,...,5D-CEMBA180605-5D-3-CEMBA180605-5D-4-G10,0.674676,0.667709,G10,CEMBA180605_5D_4,9,6,18,13,CEMBA180605_5D
9938,5D_M_997,/gale/raidix/rdx-4/CEMBA_RS1/5D/CEMBA180605_5D...,0.00478,0.79146,0.79046,0.02040,0.01570,625473.0,1365894,928844.0,...,5D-CEMBA180605-5D-3-CEMBA180605-5D-4-G10,0.673389,0.680026,G10,CEMBA180605_5D_4,9,6,19,13,CEMBA180605_5D
9939,5D_M_998,/gale/raidix/rdx-4/CEMBA_RS1/5D/CEMBA180605_5D...,0.00571,0.82569,0.82469,0.03079,0.02522,1536923.0,3652902,2506658.0,...,5D-CEMBA180605-5D-3-CEMBA180605-5D-4-G11,0.613136,0.686210,G11,CEMBA180605_5D_3,10,6,20,12,CEMBA180605_5D


In [64]:
ko.columns

Index(['cell', 'AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate',
       'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region',
       'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96',
       'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'Batch'],
      dtype='object')

In [62]:
ko['cell_id']

KeyError: 'cell_id'

# Estimate splice junctions from SMARTseq scRNA

In [79]:
#Load metadata from pachter paper
#load in metadata 
os.chdir('/cndd3/dburrows/DATA/public_datasets/splice//smartseq_mouse_p53-59_MOp_pachter21')
meta = sc.read_h5ad('isoform.h5ad')
meta

/home/dburrows/mambaforge/envs/base_conda/lib/python3.9/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/dburrows/mambaforge/envs/base_conda/lib/python3.9/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 6160 × 69172
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts', 'leiden', 'louvain'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'gene_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'louvain', 'neighbors'
    obsm: 'X_nca', 'X_nca_tsne', 'X_pca', 'X_pca_tsne'
    layers: 'X', 'log1p', 'norm'
    obsp: 'distances', 'connectivities'

In [114]:
#Grab data from nemo
from bs4 import BeautifulSoup

# Specify the path to your HTML file
file_path = '/cndd3/dburrows/DATA/public_datasets/splice/smartseq_mouse_p53-59_MOp_pachter21/nemo.html'

# Open the file and read its contents
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
fastq_links = [a['href'] for a in soup.find_all('a', href=True) if 'fastq' in a['href']]

filt_id = meta.obs['cell_id'].values
raw_id = np.asarray([i.split('.')[0] for i in fastq_links])
if sum(np.in1d(filt_id,raw_id)) == len(filt_id): print('all labelled cells are in nemo archive')

all labelled cells are in nemo archive


In [140]:
#Use filt_id to batch wget files from nemo

#generate txt file with paths
sub_str = 'https://data.nemoarchive.org/biccn/lab/zeng/transcriptome/scell/SSv4/mouse/raw/MOp/'
path_l = [sub_str + i for i in np.asarray(fastq_links)[np.in1d(raw_id, filt_id)]]

file_path = '/cndd3/dburrows/DATA/public_datasets/splice/smartseq_mouse_p53-59_MOp_pachter21/metadata/download_list.txt'

# Writing the URLs to the file
with open(file_path, 'w') as file:
    for url in path_l:
        file.write(url + '\n')
  
# batch download
#! xargs -n 1 -P 10 wget < /cndd3/dburrows/DATA/public_datasets/splice/smartseq_mouse_p53-59_MOp_pachter21/metadata/download_list.txt